In [2]:
import numpy as np
import pandas as pd
import sys, os, copy, cv2
sys.path.append('/home/rahim/Desktop/projects/mrob/build/mrobpy/')
import mrob
import matplotlib.pyplot as plt
import open3d as o3d
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
fx=918.7847900390624
fy=918.74755859375
cx=959.554443359375
cy=553.0713500976563

camera_matrix_azure = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
dist_coeff_azure = np.zeros(5)

In [4]:
root_dir = '/home/rahim/patata/datasets/navigine/mocab/apriltag/Azure/mocap'

In [5]:
def read_associations(root_dir, idx):
    with open(root_dir+'/associations.txt') as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]

    associations = []

    for i in lines:
        associations.append(i.split(" ")[1::2])

    return associations[idx-1]
read_associations(root_dir,400)

['/color_pr/000027006844.png', '/depth_pr/000027006866.png']

In [6]:
def quat_to_SE3(quat_pose):
    
    rot_4x1 = quat_pose[-4:]
    tra_3x1 = quat_pose[1:4]

    rot = mrob.geometry.quat_to_so3(rot_4x1)
    pose = mrob.geometry.SE3(mrob.geometry.SO3(rot),tra_3x1)

    return pose

In [7]:
def ax(pose, size = 0.1):
    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    return axis.transform(pose)

In [11]:
mocap_poses = pd.read_csv(root_dir+'/Azure_mocap_apriltag_color_gt.csv', header=None, sep="\s+")
mocap_quat = np.array([each for _,each in mocap_poses.iterrows()])

# orb_poses = pd.read_csv(root_dir+'/CameraTrajectory.txt', header=None, sep="\s+")
# orb_quat = np.array([each for _,each in orb_poses.iterrows()])

mocap = np.zeros((mocap_quat.shape[0],4,4))
# orb = np.zeros((mocap_quat.shape[0],4,4))
for i in range(len(mocap_quat)):
    mocap[i,:,:] = (quat_to_SE3(mocap_quat[i,:])).T()
    # orb[i,:,:] = (quat_to_SE3(orb_quat[i,:])).T()



In [8]:
# !pip install opencv-contrib-python

In [8]:
def get_data(root_dir, idx):

    pairs = read_associations(root_dir,idx)
    color_dir = root_dir + pairs[0]
    depth_dir = root_dir + pairs[1]
    
    color = o3d.t.io.read_image(color_dir)
    depth = o3d.t.io.read_image(depth_dir)
    return color, depth
c, d = get_data(root_dir, 561)

In [9]:
def get_pcd_pose(root_dir, camera_matrix, idx):

    color, depth = get_data(root_dir, idx)

    cam = o3d.camera.PinholeCameraIntrinsic()
    cam.intrinsic_matrix = camera_matrix

    rgbd = o3d.t.geometry.RGBDImage(color, depth)
                                   
    pcd = o3d.t.geometry.PointCloud.create_from_rgbd_image(rgbd, camera_matrix, depth_scale=1000.0, depth_max=3.0)
    pcd = pcd.to_legacy()

    return pcd

pcd = get_pcd_pose(root_dir, camera_matrix_azure, 541)
pcd

PointCloud with 152611 points.

In [11]:

def main(root_dir, camera_matrix, dist_coeff, idx):

    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_100)
    # Note: Pattern generated using the following link
    # https://calib.io/pages/camera-calibration-pattern-generator
    board = cv2.aruco.CharucoBoard_create(14, 9, 0.03975, 0.02975, aruco_dict)
    # (columns:squaresX, rows:squaresY, Checker Width: squareLength, markerLength, markers type)

    pairs = read_associations(root_dir,idx)
    color_dir = root_dir + pairs[0]

    all_corners, all_ids, T_arucos = [], [], []
    frame = cv2.imread(color_dir)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict)
    if len(corners) > 0:
        ret, c_corners, c_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board)
        if ret > 0:
            all_corners.append(c_corners)
            all_ids.append(c_ids)
    imsize = gray.shape

    all_corners = [x for x in all_corners if len(x) >= 4]
    all_ids = [x for x in all_ids if len(x) >= 4]
    ret, _, _, rvec, tvec = cv2.aruco.calibrateCameraCharuco(
        all_corners, all_ids, board, imsize, None, None)

    ret, p_rvec, p_tvec = cv2.aruco.estimatePoseCharucoBoard(charucoCorners=c_corners,
                                                                charucoIds=c_ids,
                                                                board=board,
                                                                cameraMatrix=camera_matrix,
                                                                distCoeffs=dist_coeff, 
                                                                rvec=0*rvec[0], 
                                                                tvec=0*tvec[0],
                                                                )
    # if ret>0:                                                           
    #     ax = cv2.drawFrameAxes(frame,
    #                             camera_matrix,
    #                             dist_coeff,
    #                             p_rvec,
    #                             p_tvec,
    #                             0.1)
    #     plt.figure(figsize=(22,18)) 
    #     plt.imshow(ax) 
    #     plt.savefig(f'/home/r/Desktop/new_project_dir/mrob/figs/{os.path.basename(pairs[0])}')                      
    #     # plt.show()
    #     plt.figure().clear()
    #     plt.close()

    T_aruco = mrob.geometry.SE3(mrob.geometry.SO3(p_rvec),p_tvec)
    return T_aruco, pairs[0]



$$^{m}T_{af}^{-1} \cdot ^{m}T_f = ^{af}T_a \cdot ^{c}T_a^{-1} \cdot ^{c}T_f$$

$$^{af}T_f = Y \cdot ^{a}T_c \cdot X$$

In [12]:
import cv2
AT_C = []
MT_F = []
pcds = []


for idx in tqdm(range(0, 1000, 10)):
    pairs = read_associations(root_dir,idx)
    try:
        cT_a, name = main(root_dir, camera_matrix_azure, dist_coeff_azure, idx)
        pcd = get_pcd_pose(root_dir, camera_matrix_azure, idx)
        # we need aT_c in formula
        AT_C.append(cT_a.inv())
        MT_F.append(mocap[idx-1])
        pcds.append(pcd)
    except: pass

100%|██████████| 100/100 [01:40<00:00,  1.01s/it]


In [12]:
mT_af_q = [0.0, -0.774173, 1.020377, 2.115314, 0.004979, 0.846904, -0.531174, 0.024143]
mT_af = quat_to_SE3(mT_af_q)
AFT_M = mT_af.inv()

In [13]:
AFT_F = []
for mT_f in MT_F:
    AFT_F.append(AFT_M.mul(mrob.geometry.SE3(mT_f)))

NameError: name 'MT_F' is not defined

In [15]:
AFT_F_ax=[ax(AFT_F[i].T(), 0.05) for i in range(len(AFT_F))]
AT_C_ax=[ax(AT_C[i].T(), 0.1) for i in range(len(AT_C))]
# ax_list.append(ax(mT_af.T(), 0.3))

In [ ]:
o3d.visualization.draw_geometries(AFT_F_ax+AT_C_ax)

### in image below, up are poses from mocap and down are poses from ruco-marker. we used opeencv for that. as shown in abowe.

![Getting Started](./Azure_mocap_poses_before_calibration.png)

$$^{af}T_m \cdot ^{m}T_f = ^{af}T_a \cdot ^{a}T_c \cdot ^{c}T_f$$

$$^{af}T_f = Y \cdot ^{a}T_c \cdot X$$

as defined in the factor:
    $$T_{obs2} =  Y_{origin} \cdot T_{obs1} \cdot X_{target}  $$

In [16]:
def solver(gt, orb):

    gt = np.asarray([i.T() for i in gt])
    orb = np.asarray([i.T() for i in orb])
    Y_ini = mrob.geometry.SE3() #gt[0,:,:]).inv()
    X_ini = mrob.geometry.SE3()

    graph = mrob.FGraph()
    W_0 = 1e0*np.identity(6)
    W = np.identity(6)
    iy = graph.add_node_pose_3d(Y_ini)
    ix = graph.add_node_pose_3d(X_ini)
    #graph.add_factor_1pose_3d(X_ini,ix,W_0)
    #graph.add_factor_1pose_3d(Y_ini,iy,W_0)
    graph.print(True)
    results = graph.get_estimated_state()
    # print('Initial conditions:\n',results[0],'\n',results[1])

    # adding all poses or some of them. Residual is r = Ln ( T_o * T_obs * T_t * T_obs2^-1 )
    # There is aproblem with the second observation... but for gradients
    for i in range(len(gt)):
        T_gt = mrob.geometry.SE3(gt[i,:,:])
        T_orb = mrob.geometry.SE3(orb[i,:,:])
        graph.add_factor_2poses_3d_2obs(obs = T_gt,
                                obs2 = T_orb,
                                nodeOriginId = iy,
                                nodeTargetId = ix,
                                obsInvCov = W)

    graph.solve(mrob.LM, 100, solutionTolerance=1e-6)

    results = graph.get_estimated_state()
    
    # np.save('results.npy', results)

    # afY_a = results[0] 
    # fX_c = results[1]

    # return afY_a, fX_c
    return results


results = solver(AT_C, AFT_F)
print('#'*30)
print('Y= \n', results[0],'\nX=\n',results[1])

Status of graph:  Nodes = 2, Factors = 0, Eigen Factors = 0
Printing NodePose3d: 0, state = 
-0
 0
-0
 0
 0
 0,
 SE3 matrix: 
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
Printing NodePose3d: 1, state = 
-0
 0
-0
 0
 0
 0,
 SE3 matrix: 
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1

FGraphSolve::optimize_levenberg_marquardt: iteration 1##############################
Y= 
 [[-0.99971456 -0.02228952  0.00860053  0.28782298]
 [-0.00816474 -0.01956904 -0.99977517 -0.03647737]
 [ 0.02245281 -0.99956002  0.01938146  0.25678577]
 [ 0.          0.          0.          1.        ]] 
X=
 [[ 0.99974679 -0.0033099   0.02225774 -0.06345485]
 [-0.00326866 -0.99999287 -0.00188887 -0.03740818]
 [ 0.02226383  0.00181564 -0.99975048 -0.22397985]
 [ 0.          0.          0.          1.        ]]
 lambda = 1e-05, error 262.781, and delta = -71.5976

FGraphSolve::optimize_levenberg_marquardt: iteration 2 lambda = 2e-05, error 262.781, and delta = -71.5926

FGraphSolve::optimize_levenberg_marquardt: iteration 3 lambda = 4e-05, error

In [17]:
# help(mrob.FGraph.add_factor_2poses_3d_2obs)

CT_F = mrob.geometry.SE3(results[1])
FT_C = CT_F.inv().T()
FT_C
# np.save('FT_C_azure', FT_C)

array([[ 0.99974679, -0.00326866,  0.02226383,  0.06830316],
       [-0.0033099 , -0.99999287,  0.00181564, -0.03721128],
       [ 0.02225774, -0.00188887, -0.99975048, -0.22258226],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [15]:
# FT_C = np.load('./FT_C_azure.npy')

In [18]:
pcds_glob = []
azure_poses = []
for pcd, mT_fi in zip(pcds, MT_F):
    mT_fi = mrob.geometry.SE3(mT_fi)
    mT_c = mT_fi * CT_F.inv()
    pcds_glob.append(pcd.transform(mT_c.T()))
    azure_poses.append(mT_c.T())


In [20]:
o3d.visualization.draw_geometries(pcds_glob)

![Getting Started](./calibration_results.png)

so we found calibration. what we do:

take pose from mocap($^{M}T_F$), and  multiply it to inverse of $X$, again $X$ here based on our chain, is $^{C}T_{F}$, frame to camera, and this multiplication will give you pose from Azure camera to mocap, IOW, global pose.

$$^{M}T_C = ^{M}T_F \cdot X^{-1} = ^{M}T_F \cdot ^{C}T_F^{-1}$$



In [12]:
pcds_all = [get_pcd_pose(root_dir, camera_matrix_azure, idx) for idx in range(0, len(mocap), 50)]

In [15]:
# FT_C = np.load('./FT_C_azure.npy')
# CT_F = mrob.geometry.SE3(FT_C).inv()

In [16]:
pcds_map = []
azure_pos = []
for pcd, mT_fi in zip(pcds_all, mocap[::50]):
    mT_fi = mrob.geometry.SE3(mT_fi)
    mT_c = mT_fi * CT_F.inv()
    pcds_map.append(copy.deepcopy(pcd).transform(mT_c.T()))
    azure_pos.append(mT_c.T())


array([[-0.997924  ,  0.03902632, -0.05123119, -0.75173797],
       [-0.0173995 , -0.9292774 , -0.36897259,  1.29246891],
       [-0.06200763, -0.36731521,  0.92802726,  0.97458113],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [19]:
o3d.visualization.draw_geometries(pcds_map[5:]+[ax(i) for i in azure_pos])

![Getting Started](./azure_map.png)

In [21]:
# lab_cloud = o3d.geometry.PointCloud()
# for pcd in pcds_map[5:]:
#     points = np.concatenate((np.asarray(lab_cloud.points), np.asarray(pcd.points)), axis=0)
#     colors = np.concatenate((np.asarray(lab_cloud.colors), np.asarray(pcd.colors)), axis=0)
#     lab_cloud.points = o3d.utility.Vector3dVector(points)
#     lab_cloud.colors = o3d.utility.Vector3dVector(colors)

# o3d.io.write_point_cloud('lab_cloud.pcd', lab_cloud)

True